<a href="https://colab.research.google.com/github/stwater20/AIS3-2024-Material/blob/main/AIS3_Lab6_Malware_API_Sequence_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ls -al

total 159744
drwxr-xr-x 1 root root      4096 Jul 25 08:20 .
drwxr-xr-x 1 root root      4096 Jul 25 08:18 ..
drwxr-xr-x 4 root root      4096 Jul 23 13:21 .config
-rw-r--r-- 1 root root 160032752 Oct 28  2021 data_demo.npz
-rw-r--r-- 1 root root   3525445 Jul 25 08:20 data_demo.zip
drwxr-xr-x 1 root root      4096 Jul 23 13:21 sample_data


In [1]:
!unzip data_demo.zip

Archive:  data_demo.zip
  inflating: data_demo.npz           


In [2]:
import numpy as np

# 讀取 npz 檔案
data = np.load('data_demo.npz')

# 列出檔案中所有的陣列
print(list(data.keys()))

['x_name', 'x_semantic', 'y']


In [3]:
data['x_name']

array([[  1,   4,  59, ...,   0,   0,   0],
       [ 10,  11,  10, ...,   0,   0,   0],
       [  1,   4,  59, ...,   0,   0,   0],
       ...,
       [123, 253, 253, ...,   0,   0,   0],
       [ 15,   3,   3, ...,  27,  27,  27],
       [  1,  10,  11, ...,   9,  76,  52]])

In [4]:
len(data['x_name'][0]) # 每個軟體有 1000 個 API 序列

1000

In [5]:
data['x_semantic']

array([[  4, 101,  21, ...,  14,  14,  14],
       [  0,  25, 255, ...,  14,  14,  14],
       [  4, 101,  21, ...,  14,  14,  14],
       ...,
       [  0,   1, 163, ...,  14,  14,  14],
       [  0,   5,  61, ...,  14,  32,  33],
       [  4, 101,  21, ...,   5,  79,  80]])

In [6]:
len(data['x_semantic'][0])

4000

In [ ]:
data['y'],len(data['y'])

(array([[0],
        [0],
        [0],
        ...,
        [1],
        [1],
        [1]]),
 4000)

In [ ]:
flat_list = [item for sublist in data['y'].tolist() for item in sublist]

In [ ]:
from collections import Counter
Counter(flat_list)

Counter({0: 2000, 1: 2000})

In [ ]:
import sys
import os.path as osp
from random import random
import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import Embedding, Conv1d, LSTM, Linear, BCELoss, Conv2d, ZeroPad2d
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import pandas as pd
import torch.utils.data as Data

import warnings

warnings.filterwarnings("ignore")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print('device', device)


class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.embedder1 = Embedding(num_embeddings=316, embedding_dim=16)
        self.embedder2 = Embedding(num_embeddings=289, embedding_dim=8)

        self.cnn1_1 = Conv2d(in_channels=1, out_channels=128, kernel_size=(3, 16), stride=1, padding=(1, 0))
        self.cnn1_2 = Conv2d(in_channels=1, out_channels=128, kernel_size=(4, 16), stride=1)
        self.cnn1_3 = Conv2d(in_channels=1, out_channels=128, kernel_size=(5, 16), stride=1, padding=(2, 0))

        self.cnn2 = Conv2d(in_channels=1, out_channels=128, kernel_size=(4, 8), stride=4)

        self.lstm = LSTM(input_size=512, hidden_size=100, bidirectional=True, batch_first=True)

        self.lin1 = Linear(200, 64)
        self.lin2 = Linear(64, 32)
        self.lin3 = Linear(32, 1)

    def forward(self, data):
        x_name, x_behavior = data.split([1000, 4000], 1)

        # print(x_name.shape)
        # print(x_behavior.shape)

        x_name = self.embedder1(x_name)
        x_behavior = self.embedder2(x_behavior)

        x_name = x_name.unsqueeze(1)
        x_behavior = x_behavior.unsqueeze(1)

        pad = ZeroPad2d(padding=(0, 0, 2, 1))
        x_name_pad = pad(x_name)

        x_name_cnn1 = F.relu(self.cnn1_1(x_name)).squeeze(-1).permute(0, 2, 1)
        x_name_cnn2 = F.relu(self.cnn1_2(x_name_pad)).squeeze(-1).permute(0, 2, 1)
        x_name_cnn3 = F.relu(self.cnn1_3(x_name)).squeeze(-1).permute(0, 2, 1)

        x_behavior = F.relu(self.cnn2(x_behavior)).squeeze(-1).permute(0, 2, 1)

        x = torch.cat([x_name_cnn1, x_name_cnn2, x_name_cnn3, x_behavior], dim=-1)

        x, (h_n, c_n) = self.lstm(x)

        output_fw = h_n[-2, :, :]
        output_bw = h_n[-1, :, :]

        x = torch.cat([output_fw, output_bw], dim=-1)
        x = F.relu(self.lin1(x))
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(self.lin2(x))
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.sigmoid(self.lin3(x))

        return x


def train(epoch, loader):
    model.train()

    loss_all = 0
    for step, (b_x, b_y) in enumerate(loader):
        b_x = b_x.to(device)
        b_y = b_y.to(device)
        optimizer.zero_grad()
        output = model(b_x)
        loss = F.binary_cross_entropy(output, b_y)
        loss.backward()
        loss_all += b_x.size(0) * loss.item()
        optimizer.step()
    return loss_all / len(train_x)


def test(loader):
    model.eval()
    label_list = []
    pred_list = []
    for step, (b_x, b_y) in enumerate(loader):
        b_x = b_x.to(device)
        pred = model(b_x)
        pred = torch.where(pred >= 0.5, torch.ones_like(pred), torch.zeros_like(pred))

        label_list_batch = b_y.to('cpu').detach().numpy().tolist()
        pred_list_batch = pred.to('cpu').detach().numpy().tolist()
        for label_item in label_list_batch:
            label_list.append(label_item)
        for pred_item in pred_list_batch:
            pred_list.append(pred_item)

    y_true = np.asarray(label_list)
    y_pred = np.asarray(pred_list)
    _val_confusion_matrix = confusion_matrix(y_true, y_pred)
    _val_acc = accuracy_score(y_true, y_pred)
    _val_precision = precision_score(y_true, y_pred)
    _val_recall = recall_score(y_true, y_pred)
    _val_f1 = f1_score(y_true, y_pred)
    return _val_confusion_matrix, _val_acc, _val_precision, _val_recall, _val_f1


'''
data = np.load('./data/detection_train_test.npz', allow_pickle=True)
train_x_name = data['train_x_name']
train_x_semantic = data['train_x_semantic']
train_y = data['train_y']
test_x_name = data['test_x_name']
test_x_semantic = data['test_x_semantic']
test_y = data['test_y']
train_x = np.concatenate([train_x_name, train_x_semantic], 1)
test_x = np.concatenate([test_x_name, test_x_semantic], 1)
'''

## the whole dataset is too big, use the demo dataset
data = np.load('./data_demo.npz')
train_x_name = data['x_name']
train_x_semantic = data['x_semantic']
train_y = data['y']
test_x_name = data['x_name']
test_x_semantic = data['x_semantic']
test_y = data['y']
train_x = np.concatenate([train_x_name, train_x_semantic], 1)
test_x = np.concatenate([test_x_name, test_x_semantic], 1)

train_xt = torch.from_numpy(train_x)
test_xt = torch.from_numpy(test_x)
train_yt = torch.from_numpy(train_y.astype(np.float32))
test_yt = torch.from_numpy(test_y.astype(np.float32))

train_data = Data.TensorDataset(train_xt, train_yt)
test_data = Data.TensorDataset(test_xt, test_yt)

train_loader = Data.DataLoader(
    dataset=train_data,
    batch_size=128,
    shuffle=True,
    num_workers=1,
)

test_loader = Data.DataLoader(
    dataset=test_data,
    batch_size=128,
    num_workers=1,
)

model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train_scores = {'confusion_matrix': [], 'acc': [], 'precision': [], 'recall': [], 'f1': []}
test_scores = {'confusion_matrix': [], 'acc': [], 'precision': [], 'recall': [], 'f1': []}

from tqdm import tqdm
# train
for epoch in tqdm(range(1, 61)):
    loss = train(epoch, train_loader)
    con, acc, precision, recall, f1 = test(train_loader)
    print(
        'Epoch: {:03d}, Loss: {:.5f}, Train Acc: {:.5f}, Train Precision: {:.5f}, Train Recall: {:.5f}, Train F1: {:.5f}'.
            format(epoch, loss, acc, precision, recall, f1))
    train_scores['confusion_matrix'].append(con)
    train_scores['acc'].append(acc)
    train_scores['precision'].append(precision)
    train_scores['recall'].append(recall)
    train_scores['f1'].append(f1)

# test
con, acc, precision, recall, f1 = test(test_loader)
test_scores['confusion_matrix'].append(con)
test_scores['acc'].append(acc)
test_scores['precision'].append(precision)
test_scores['recall'].append(recall)
test_scores['f1'].append(f1)
print('Epoch: {:03d}, Loss: {:.5f}, Test Acc: {:.5f}, Test Precision: {:.5f}, Test Recall: {:.5f}, Test F1: {:.5f}'.
      format(epoch, loss, acc, precision, recall, f1))

# torch.save(model, './model/model.pkl')

device cuda


  2%|▏         | 1/60 [00:12<11:51, 12.06s/it]

Epoch: 001, Loss: 0.49895, Train Acc: 0.84625, Train Precision: 0.78723, Train Recall: 0.94900, Train F1: 0.86058


  3%|▎         | 2/60 [00:21<10:23, 10.75s/it]

Epoch: 002, Loss: 0.30158, Train Acc: 0.90450, Train Precision: 0.89892, Train Recall: 0.91150, Train F1: 0.90516


...

 98%|█████████▊| 59/60 [10:19<00:10, 10.67s/it]

Epoch: 059, Loss: 0.01564, Train Acc: 0.98200, Train Precision: 0.99742, Train Recall: 0.96650, Train F1: 0.98172


100%|██████████| 60/60 [10:30<00:00, 10.50s/it]

Epoch: 060, Loss: 0.02701, Train Acc: 0.99200, Train Precision: 0.99200, Train Recall: 0.99200, Train F1: 0.99200


Epoch: 060, Loss: 0.02701, Test Acc: 0.99200, Test Precision: 0.99200, Test Recall: 0.99200, Test F1: 0.99200


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import torch.nn.functional as F
from torch.nn import Embedding, LSTM, Linear, BCELoss
from sklearn.metrics import f1_score, precision_score, recall_score
import numpy as np
import warnings

warnings.filterwarnings("ignore")

class LSTMNet(torch.nn.Module):
    def __init__(self):
        super(LSTMNet, self).__init__()
        self.embedder = Embedding(num_embeddings=316, embedding_dim=16)
        self.lstm = LSTM(input_size=16, hidden_size=100, bidirectional=True, batch_first=True)
        self.lin1 = Linear(200, 64)
        self.lin2 = Linear(64, 1)

    def forward(self, x):
        x = self.embedder(x)
        x, (h_n, c_n) = self.lstm(x)
        x = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)
        x = F.relu(self.lin1(x))
        x = torch.sigmoid(self.lin2(x))
        return x

def test(model, device, test_loader):
    model.eval()
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            preds = output.round()
            all_preds.extend(preds.view(-1).cpu().numpy())
            all_targets.extend(target.cpu().numpy())

    # Calculate metrics
    precision = precision_score(all_targets, all_preds)
    recall = recall_score(all_targets, all_preds)
    f1 = f1_score(all_targets, all_preds)

    return precision, recall, f1

## the whole dataset is too big, use the demo dataset
data = np.load('./data_demo.npz')
train_x_name = data['x_name']
train_y = data['y']
test_x_name = data['x_name']
test_y = data['y']

train_xt = torch.from_numpy(train_x_name).long()
test_xt = torch.from_numpy(test_x_name).long()
train_yt = torch.from_numpy(train_y.astype(np.float32))
test_yt = torch.from_numpy(test_y.astype(np.float32))

train_data = TensorDataset(train_xt, train_yt)
test_data = TensorDataset(test_xt, test_yt)

train_loader = DataLoader(dataset=train_data, batch_size=128, shuffle=True, num_workers=1)
test_loader = DataLoader(dataset=test_data, batch_size=128, num_workers=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = LSTMNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
for epoch in range(1, 11):  # 10 epochs
    model.train()
    total_loss = 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        if target.dim() == 1:
            target = target.unsqueeze(1)  # target 現在應該是 [batch_size, 1]
        loss = BCELoss()(output, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch}, Train Loss: {total_loss / len(train_loader)}')



# Test
precision, recall, f1 = test(model, device, test_loader)
print(f'Test Precision: {precision:.4f}, Test Recall: {recall:.4f}, Test F1: {f1:.4f}')


Epoch 1, Train Loss: 0.49372848123311996
Epoch 2, Train Loss: 0.35968773625791073
Epoch 3, Train Loss: 0.35932787135243416
Epoch 4, Train Loss: 0.3494661985896528
Epoch 5, Train Loss: 0.39521539211273193
Epoch 6, Train Loss: 0.32728859363123775
Epoch 7, Train Loss: 0.2686572172679007
Epoch 8, Train Loss: 0.2746171588078141
Epoch 9, Train Loss: 0.23142848117277026
Epoch 10, Train Loss: 0.20562454778701067
Test Precision: 0.9007, Test Recall: 0.9430, Test F1: 0.9213
